# Handwritten Digit Recognition using k-nearest neighbors
## Lena Fleischhacker, Pia Röhrich, Hellen Röttgen, Benjamin Wehnert
### July 2022

## Abstract

## Table of contents

## 1) Introduction
The main goal of the project is to write an algorithm that accurately recognizes handwritten digits from the MNIST dataset using the K-nearest neighbors method. Digit recognition is a form of pattern recognition which describes the operation of identifying digits from images. This is not an easy task, taking into consideration that handwritten numbers are not written in the same style. Thus, the categories are harder to classify. With almost all processes in everyday life being digitalized, e.g. banking transactions or contact forms, developing such algorithms has become extremely useful and almost inevitable.

For this reason, the MNIST data set has been well studied and analyzed over the past decades, leading to the development of algorithms that recognize digits with the same accuracy as human beings. This laid the foundation for the development of more advanced pattern recognition algorithms such as detecting house numbers from photos with the SVHN (Street View house numbers) dataset. These algorithms can be very useful for future applications like better navigation systems.

However, in order to properly understand the fundmentals of machine learning, it is useful to reproduce the basic steps of digit recognition using the MNIST data set.

Additionally, an analysis of the accuracy of recognition with the KNN method is going to be part of the project.

In the last part of the project, we looked for a creative application of the data set


### 1.1.) Material (information data set)??
The data set that we used for the project is the standardized MNIST data set. MNIST stands for "Modified National Institute of Standard and Technology database". The data set consists of 70000 images which are divided into a training data set comprising 60000 images and a test data set comprising 10000 images. Each image contains a handwritten digit ranging from zero to nine. 

The images are stored as CSV (comma-seperated values) files. Each image consists of 28 x 28 = 784 pixels with intensity values ranging from 0 to 255. This means that all training images can be displayed in a 60000 x 785 matrix with the first column giving the actual number displayed in the image. Consequently, the test images can be dispalyed as a 10000 x 784 matrix.


In [ ]:
#codeprinting normal numbers

### 1.2.) Average images
To get a first impression of the images in the data set, we wrote an algorithm calculating mean digits. 
For this, we used all images from the training data set containing the same number. We then summed up the intensitied for each pixel an divided that values by the number of images used. Surprisingly, the numbers were still recognizable.  

This simple operation also enabled us to detect numbers with an accuracy of ... by simply comparing the image to the ten average images.

In [ ]:
#code printing average images

## 2) Methods

### 2.1.) Z-Tranformation (PCA preparation)
First of all, the data is z-transformed for easier handling. Using the z-transformation, values of the sample are converted to z-scores. The distribution of these z-scores has a mean of 0 and a standard deviation of 1. Z-scores make the sample comparable by making the data non-dimensional. The z-scores are calculated as follows:

(keine Ahnung ob die funktioniert, ist erstmal nur von letztem Jahr kopiert!!)
$$\\text{cov}(x,y) = \\frac{1}{N - 1} \\cdot \\sum_{i=1}^{N} \\left(X_i - \\overline{X}\\right)\\left(Y_i - \\overline{Y}\\right)$$\n"

Applied to the project, this means that the average intensity and standard deviation for each pixel need to be calculated. After that, each pixel in each of the images needs to be z-transformed.

In [ ]:
#code z-transformation (one line)

### 2.2.) PCA
Before implementing the Knn algorithm, we implemented a PCA to reduce dimensionality. This is a very important for a shorter run time of the code.

#### 2.2.1.) Visualization of PCA


In [ ]:
#code PCA visualization

### 2.3.) KNN
K-nearest neighbors (KNN) is the machine learning algorithm that we are using to classify the numbers. It is a simple, supervised machine learning algorithm.

### 2.4.) methods used for creative application

## Results

### result transformeirte Bilder
### resulte PCA
### result knn
### result creative application

## Discussion

## References
Netzer, Y. et al. "Reading Digits in Natural Images with Unsupervised Feature Learning." Proceedings of the Workshop on Neural Information Processing Systems (2011)